# ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile)
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821



## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    
# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS Music
    WITH REPLICATION={'class':'SimpleStrategy','replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('music')
except Exception as e:
    print(e)

### Gives the artist, song title and song's length in the music app.
- #### sessionId and iteminSession are considered as COMPOSITE KEY.

In [8]:
#Create Table Music_app 
query="CREATE TABLE Music_app "
query = query+"(sessionId int,itemInSession int, artist text, song_title text, length decimal, PRIMARY KEY(sessionId,itemInSession));"
try:
    session.execute(query)
except Exception as e:
    print(e)         

### Insert data into the tables from event_datafile_new.csv file

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#insert data into Music_app
        query = "INSERT INTO Music_app(sessionId, itemInSession, artist, song_title, length)"
        query = query + " VALUES(%s,%s,%s,%s,%s);"
## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### A SELECT Statemnt to verify that the data have been inserted into each table

In [10]:
query="select artist, song_title, length from music_app where sessionId = 338 and itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print("artist: ",row.artist, "Song Title: ",row.song_title, "Length: ",row.length)

artist:  Faithless Song Title:  Music Matters (Mark Knight Dub) Length:  495.3073


## Gives name of artist, song (sorted by itemInSession) and user (first and last name)
- userid and sessionid are considered as the COMPOSITE KEY
- itemInSession is the CLUSTERING COLUMN for Sorting.

In [11]:
#Create Table Session
query="CREATE TABLE session "
query = query+"(sessionId int,itemInSession int, artist text, song_title text, userid int,user text, PRIMARY KEY((userid, sessionid),itemInSession));"
try:
    session.execute(query)
except Exception as e:
    print(e)    

### Insert data into the tables from event_datafile_new.csv file

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# INSERT data into session
        query = "INSERT INTO session(sessionId, itemInSession, artist, song_title, userid, user)"
        query = query + " VALUES(%s,%s,%s,%s,%s,%s);"
## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], int(line[10]), line[1]+' '+line[4]))

#### A SELECT Statement to verify that the data have been inserted into each table

In [13]:
query="select artist, song_title, user from session where userid = 10 and sessionid = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print("artist: ",row.artist, "Song Title: ",row.song_title, "User Name: ",row.user)

artist:  Down To The Bone Song Title:  Keep On Keepin' On User Name:  Sylvie Cruz
artist:  Three Drives Song Title:  Greece 2000 User Name:  Sylvie Cruz
artist:  Sebastien Tellier Song Title:  Kilometer User Name:  Sylvie Cruz
artist:  Lonnie Gordon Song Title:  Catch You Baby (Steve Pitron & Max Sanna Radio Edit) User Name:  Sylvie Cruz


## Gives every user name (first and last) in my music app history who listened to a song
- song_title and userid are considered as the COMPOSITE KEY

In [14]:
#create Table Music_app_History
query="CREATE TABLE Music_app_history "
query = query+"(userId int,user text, gender text, song_title text, PRIMARY KEY(song_title,userid));"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Insert data into the tables from event_datafile_new.csv file

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT statements into the `query` variable
        query = "INSERT INTO Music_app_history(userId, user, gender, song_title)"
        query = query + " VALUES(%s,%s,%s,%s);"
## Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), line[1]+' '+line[4], line[2], line[9]))

#### A SELECT to verify that the data have been inserted into each table

In [16]:
query="select user from Music_app_history where song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print("User Name: ",row.user)

User Name:  Jacqueline Lynch
User Name:  Tegan Levine
User Name:  Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## Drop the table before closing out the sessions
query = "Drop table Music_app"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "Drop table session"

try:
    session.execute(query)
except Exception as e:
    print(e)

query = "Drop table Music_app_history"

try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()